# Priority Queue

In [43]:
:ext GADTs
:ext ScopedTypeVariables
:ext StandaloneDeriving

data Queue a where
  Queue :: (Show a, Ord a) => Int -> a -> Queue a -> Queue a -> Queue a 
  Empty :: (Show a, Ord a) => Queue a 
deriving instance Show (Queue a)

merge :: forall a. Queue a -> Queue a -> Queue a
merge Empty q = q
merge q Empty = q
merge left@(Queue _ lv ll lr) right@(Queue _ rv rl rr)
  | lv <= rv = swap lv ll (merge lr right)
  | otherwise = swap rv rl (merge rr left)
  where
    -- this is to preserve the 'leftist' property: the larger subtree must be on the left
    swap :: a -> Queue a -> Queue a -> Queue a
    swap v l r = Queue rank v left right where
      (left, right) | getRank l >= getRank r = (l, r)
                    | otherwise = (r, l)
      rank = getRank right + 1
      
getRank :: Queue a -> Int
getRank Empty = 0
getRank (Queue r _ _ _) = r

singleton :: (Show a, Ord a) => a -> Queue a
singleton v = Queue 1 v Empty Empty

-- inserting is just merging with a singleton
insert :: (Show a, Ord a) => a -> Queue a -> Queue a
insert a = merge (singleton a)

getMin :: Queue a -> Maybe a
getMin Empty = Nothing
getMin (Queue _ v _ _) = Just v

deleteMin :: Queue a -> Queue a
deleteMin Empty = Empty
deleteMin (Queue _ _ l r) = merge l r


-- try it out
q1 = singleton 10
q1
q2 = insert 5 q1
q2
q3 = insert 20 q2
q3
q4 = insert 15 q3
q4
q5 = insert 25 q4
q5
getMin q5
deleteMin q5

Queue 1 10 Empty Empty

Queue 1 5 (Queue 1 10 Empty Empty) Empty

Queue 2 5 (Queue 1 10 Empty Empty) (Queue 1 20 Empty Empty)

Queue 2 5 (Queue 1 10 Empty Empty) (Queue 1 15 (Queue 1 20 Empty Empty) Empty)

Queue 2 5 (Queue 2 15 (Queue 1 20 Empty Empty) (Queue 1 25 Empty Empty)) (Queue 1 10 Empty Empty)

Just 5

Queue 1 10 (Queue 2 15 (Queue 1 20 Empty Empty) (Queue 1 25 Empty Empty)) Empty

## Testing with QuickCheck
Lets use `QuickCheck` to verify that our `Queue` adheres to the invariants

In [45]:
:m Test.QuickCheck

`QuickCheck` generates the tests for us. We just need to write `properties` for it to verify.

We also need arbitrary instances of `Queue`. For this we'll use the `Arbitrary` class. We'll use the `Gen` monad to make a queue from a list of values.

In [50]:
qFromList :: (Show a, Ord a) => [a] -> Gen (Queue a)
qFromList = return . foldr insert Empty

instance (Arbitrary a, Show a, Ord a) => Arbitrary (Queue a) where
  arbitrary = listOf arbitrary >>= qFromList
  
:t arbitrary
:t listOf

arbitrary :: forall a. Arbitrary a => Gen a

listOf :: forall a. Gen a -> Gen [a]

Lets first verify that the 'leftist' property holds. To do this we write a function that takes a tree and verifies that each node follows the rule. We will verify that nodes store their rank in the process.

In [60]:
verifyLeftist :: Queue a -> Bool
verifyLeftist Empty = True
verifyLeftist q@(Queue rank _ l r) =
  and [ qrank q == rank
      , qrank l >= qrank r
      , verifyLeftist l
      , verifyLeftist r
      ]
 where
   qrank Empty = 0
   qrank (Queue _ _ l r) = 1 + min (qrank l) (qrank r)

We also want to verify the heap ordering property.

In [65]:
:m Data.Maybe

verifyHeap :: Queue a -> Bool
verifyHeap Empty = True
verifyHeap q@(Queue _ _ l r) =
  and [ comp q l
      , comp q r
      , verifyHeap l
      , verifyHeap r ]
 where
   comp root child = fromMaybe True $ (<=) <$> (getMin root) <*> (getMin child)

Now we can run `quickCheck` against these functions.

In [66]:
quickCheck (verifyLeftist :: Queue Int -> Bool)
quickCheck (verifyHeap :: Queue Int -> Bool)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.